In [ ]:
!python -m pip install -r requirements.txt

In [ ]:
class Interval:
    def __init__(self, inf, sup=None):
        if (not (isinstance(inf, float) or isinstance(inf, int)) or not (
                isinstance(sup, float) or isinstance(sup, int) or sup is None)):
            raise TypeError()
        if sup is None:
            sup = inf
        if inf > sup:
            raise ValueError()
        self.inf, self.sup = float(inf), float(sup)

    def __add__(self, other):
        return Interval(self.inf + other.inf, self.sup + other.sup)

    def __mul__(self, other):
        args = (self.inf * other.inf, self.inf * other.sup,
            self.sup * other.inf, self.sup * other.sup)
        return Interval(min(args), max(args))

    def __neg__(self):
        return Interval(-self.sup, -self.inf)
    
    def __pos__(self):
        return Interval(self.inf, self.sup)

    def __repr__(self):
        return F'Interval({self.inf}, {self.sup})'

    def __str__(self):
        return F'[{self.inf}, {self.sup}]'

    def __sub__(self, other):
        return Interval(self.inf - other.sup, self.sup - other.inf)

    def __truediv__(self, other):
        if other.inf <= 0.0 and other.sup >= 0.0:
            raise ZeroDivisionError()
        return self * Interval(1.0 / other.sup, 1.0 / other.inf)

In [ ]:
import numpy as np

def gaussian_elimination(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    if not isinstance(a, np.ndarray) or not isinstance(b, np.ndarray):
        raise TypeError()
    n = len(a)
    if n < 1 or a.shape != (n, n) or b.shape != (n, ):
        raise ValueError()
    a, b = np.copy(a), np.copy(b)
    for i in range(n):
        i_max = i + np.argmax(np.abs(a[i :, i]))
        if np.abs(a[i_max, i]) < np.finfo(float).eps:
            raise ZeroDivisionError()
        if i_max != i:
            first, second = [i_max, i], [i, i_max]
            a[first], b[first] = a[second], b[second]
        for index in range(n):
            if index == i:
                continue
            mu = -a[index, i] / a[i, i]
            a[index, i:] += mu * a[i, i:]
            b[index] += mu * b[i]
    return b / np.diagonal(a)

In [ ]:
from numpy.linalg import solve

a, b = np.array([
    [2.0, 7.0],
    [3.0, 1.0]
]), np.array([9.0, 6.0])
o = solve(a, b)
assert np.allclose(gaussian_elimination(a, b), o)

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(-10, 10, 2 ** 10 + 1)
ab_line = (b[0] - x * a[0, 0]) / a[0, 1]
cd_line = (b[1] - x * a[1, 0]) / a[1, 1]

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white')
subplot.plot(x, ab_line, color='green', lw=1, label='AB')
subplot.plot(x, cd_line, color='blue', lw=1, label='CD')
subplot.plot(o[0], o[1], 'Xr', lw=1, label='AB∩CD')
plt.legend()
plt.show()

In [ ]:
interval_vectorize, epsilon = np.vectorize(Interval), Interval(0.0, 0.1)
a_interval = interval_vectorize(a) + epsilon
b_interval = interval_vectorize(b) + epsilon

print(o)
print('\n')
print(solve2(a_interval, b_interval))
print(solve(a_interval, b_interval))

In [ ]:
x = np.linspace(-10, 10, 2**10 + 1)
ab_line = (b[0] - x * a[0, 0]) / a[0, 1]
cd_line = (b[1] - x * a[1, 0]) / a[1, 1]

fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white')
subplot.plot(x, ab_line, color='green', lw=7, label='AB')
subplot.plot(x, cd_line, color='blue', lw=7, label='CD')
plt.legend()
plt.show()

# Интервальное решение
# Точное решение интервальной системы
# Итерационные методы решения слау

In [ ]:
ab = np.array([b[0] / a[0, 0], b[0] / a[0, 1])
cd = np.array([b[1] / a[1, 0], b[1] / a[1, 1])
t = np.linspace(-10, 10, 2**10 + 1)
ab_line = o + t * ab
cd_line = o + t * cd